In [ ]:
!pip install tensorflow-text
!pip install streamlit 
!pip install pyngrok==4.1.1 

     |████████████████████████████████| 4.9 MB 6.8 MB/s 
     |████████████████████████████████| 462 kB 59.9 MB/s 
     |████████████████████████████████| 10.1 MB 9.2 MB/s 
     |████████████████████████████████| 181 kB 44.7 MB/s 
     |████████████████████████████████| 111 kB 47.1 MB/s 
     |████████████████████████████████| 164 kB 45.3 MB/s 
     |████████████████████████████████| 76 kB 3.0 MB/s 
     |████████████████████████████████| 4.3 MB 51.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 131 kB 55.4 MB/s 
     |████████████████████████████████| 428 kB 58.1 MB/s 
     |████████████████████████████████| 130 kB 55.5 MB/s 
     |████████████████████████████████| 793 kB 45.6 MB/s 
     |████████████████████████████████| 381 kB 47.7 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=557341e5e86d80a0b7e982bed0aece4fdf6cebc4ab08591a429aac94678f29c2
  Stored in directory: /root/.cache/pip/whe

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=45fe1f352234683208c4372be4be9d4842af76cabbd6d0d68f24d06f8f1ecfad
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [ ]:
!gdown https://drive.google.com/uc?id=19QXIDvlHaDMj-todwfE_FPMlcaIoVNcO
!unzip saved_model_pre.zip

Downloading...
From: https://drive.google.com/uc?id=19QXIDvlHaDMj-todwfE_FPMlcaIoVNcO
To: /content/saved_model_pre.zip
100% 405M/405M [00:06<00:00, 66.5MB/s]
Archive:  saved_model_pre.zip
   creating: saved_model_pre/
   creating: saved_model_pre/my_model_bert_pre/
  inflating: saved_model_pre/my_model_bert_pre/saved_model.pb  
   creating: saved_model_pre/my_model_bert_pre/variables/
  inflating: saved_model_pre/my_model_bert_pre/variables/variables.index  
  inflating: saved_model_pre/my_model_bert_pre/variables/variables.data-00000-of-00001  
   creating: saved_model_pre/my_model_bert_pre/assets/
  inflating: saved_model_pre/my_model_bert_pre/assets/vocab.txt  
  inflating: saved_model_pre/my_model_bert_pre/keras_metadata.pb  


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

best_model = tf.keras.models.load_model('/content/saved_model_pre/my_model_bert_pre')

test_input_raw = st.text_input("Enter PubMed Abstract (Wprowadz streszczenie PubMed)")
test_input_list = test_input_raw.split('. ')

if st.button("Classify Abstract"):
  pred_state_info = st.text("Classification ongoing...")

  test_input_prob_preds = best_model.predict(test_input_list)
  test_input_preds = tf.argmax(test_input_prob_preds, axis=1)

  background_sentences = []
  conclusions_sentences = []
  methods_sentences = []
  objective_sentences = []
  results_sentences = []

  for sentence_id, sentence in enumerate(test_input_list):
    # 'BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVE', 'RESULTS'
    label_number = test_input_preds[sentence_id].numpy()

    if label_number == 0:
      background_sentences.append(sentence)
    if label_number == 1:
      conclusions_sentences.append(sentence)
    if label_number == 2:
      methods_sentences.append(sentence)
    if label_number == 3:
      objective_sentences.append(sentence)
    if label_number == 4:
      results_sentences.append(sentence)

  pred_state_info.text("CLASSIFICATION FINISHED!!!")

  st.write(f"BACKGROUND (WPROWADZENIE): {background_sentences} \n")

  st.write(f"CONCLUSIONS (WNIOSEK): {conclusions_sentences} \n")
  st.write(f"METHODS (METODY): {methods_sentences} \n")
  st.write(f"OBJECTIVE (CEL): {objective_sentences} \n")
  st.write(f"RESULTS (WYNIKI): {results_sentences} \n")

Overwriting app.py


## 1.Streamlit launcher 

In [ ]:
!ngrok authtoken ## Tutaj nalezy wprowadzic wlasny token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
## Uruchom aplikacje
!streamlit run app.py &>/dev/null&

In [ ]:
## Uruchom tunelowanie
from pyngrok import ngrok

# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(port='8501')
public_url

'http://f4a8-34-125-195-19.ngrok.io'

In [ ]:
## Check processes
!ps

In [ ]:
## Kill streamlit process
!killall streamlit

In [ ]:
## Zakoncz tunelowanie 
ngrok.kill()